In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\Live-Sports-Trader\src"))
import domain
import repository 
import pandas as pd
from dotenv import load_dotenv
from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds
import time

# Add the directory containing polymarket_functions.py to the system path
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\polymarket_sniper\Production"))
sys.path.append(os.path.abspath(r"C:\Users\Site Oracle\Live-Sports-Trader\src"))

# Now you can import the module
import polymarket_functions
import repository
from domain import Odds, Game, Team
import service



In [2]:
client = polymarket_functions.authenticate()

In [ ]:
res = polymarket_functions.get_market_positions(user_address="0x2f8b3EBBfbF52fA379c8f69e4427c7491771754f")
print(res)

In [4]:
# will need to run this once per day to update the markets.csv file
# polymarket_functions.update_markets(client)


In [5]:
res = repository.get_polymarket_team_odds_today("MLB game")

In [ ]:
res.iloc[0]['tokens']

In [7]:

games = []
import json 

for _, row in res.iterrows():
    game_name = row['question']
    game = Game(game_name)

    # Parse team names from the question (assumes format "Team1 vs. Team2")
    team_names = [name.strip() for name in game_name.split(' vs. ')]

    # Extract tokens data - handle both string and list formats
    tokens_data = row['tokens']
    if isinstance(tokens_data, str):
        # More robust parsing for Python-like string representation
        try:
            # Replace Python booleans and None with JSON equivalents
            json_str = (tokens_data
                        .replace("'", '"')
                        .replace('False', 'false')
                        .replace('True', 'true')
                        .replace('None', 'null'))
            tokens_data = json.loads(json_str)
        except json.JSONDecodeError:
            # If JSON parsing fails, try using ast.literal_eval as fallback
            import ast
            tokens_data = ast.literal_eval(tokens_data)

    # Create Team objects
    for i, token_data in enumerate(tokens_data):
        if i < len(team_names):
            team_name = team_names[i]
            token_id = token_data['token_id']
            
            team = Team(name=team_name, token_id=token_id)
            game.teams.append(team)

    games.append(game)

In [ ]:
for game in games:
    print(game.name)
    for team in game.teams:
        print(team.name)
        print(team.token_id)
        print(team.prediction_market_odds)
        print(team.true_odds)

In [ ]:
driver = repository.get_driver_fanduel_live_odds_page_existing_chrome()

live_odds = repository.parse_live_odds(driver=driver)
print(live_odds)
print(len(live_odds))

In [ ]:
driver = repository.get_driver_fanduel_live_odds_page_existing_chrome()

live_odds = repository.parse_live_odds(driver=driver)
print(live_odds)
print(len(live_odds))

In [ ]:
{'Pittsburgh Pirates': '-106', 'Chicago Cubs': '-110', 'Miami Marlins': '-102', 'Washington Nationals': '-116', 'Toronto Blue Jays': '+100', 'Philadelphia Phillies': '-118', 'Los Angeles Angels': '+154', 'Baltimore Orioles': '-184', 'Tampa Bay Rays': '+138', 'New York Mets': '-164', 'Cincinnati Reds': '+110', 'Detroit Tigers': '-130', 'New York Yankees': '+122', 'Boston Red Sox': '-144', 'Colorado Rockies': '+250', 'Atlanta Braves': '-310', 'Chicago White Sox': '+154', 'Texas Rangers': '-184', 'Minnesota Twins': '-102', 'Houston Astros': '-116', 'St. Louis Cardinals': '+126', 'Milwaukee Brewers': '-148', 'Athletics': '+134', 'Kansas City Royals': '-158', 'San Diego Padres': '+116', 'Arizona Diamondbacks': '-134', 'Cleveland Guardians': '+108', 'Seattle Mariners': '-126', 'San Francisco Giants': '+154', 'Los Angeles Dodgers': '-184'}

# WILL NEED TO UPDATE THIS MAP WHEN NEW TEAMS ARE ADDED TO FANDUEL
map_polymarket_name_to_fanduel_name = {
"Yankees": "New York Yankees",
"Red Sox": "Boston Red Sox",   
"Cardinals": "St. Louis Cardinals",
"Brewers": "Milwaukee Brewers",
"Padres": "San Diego Padres",
"Diamondbacks": "Arizona Diamondbacks",
"Marlins": "Miami Marlins",
"Nationals": "Washington Nationals",
"Guardians": "Cleveland Guardians",
"Mariners": "Seattle Mariners",
"Giants": "San Francisco Giants",
"Dodgers": "Los Angeles Dodgers",
"Blue Jays": "Toronto Blue Jays",
"Phillies": "Philadelphia Phillies",
"Twins": "Minnesota Twins",
"Astros": "Houston Astros",
"Pirates": "Pittsburgh Pirates",
"Cubs": "Chicago Cubs",
"Rays": "Tampa Bay Rays",
"Mets": "New York Mets",
"Rockies": "Colorado Rockies",
"Braves": "Atlanta Braves",
"Athletics": "Athletics",
"Royals": "Kansas City Royals",
"White Sox": "Chicago White Sox",
"Rangers": "Texas Rangers",
"Angels": "Los Angeles Angels",
"Orioles": "Baltimore Orioles",
"Reds": "Cincinnati Reds",
"Tigers": "Detroit Tigers"}

map_fanduel_name_to_polymarket_name = {v: k for k, v in map_polymarket_name_to_fanduel_name.items()}

print(map_fanduel_name_to_polymarket_name['New York Yankees'])

In [12]:
driver = repository.get_driver_fanduel_live_odds_page_existing_chrome()

In [ ]:
live_odds = repository.parse_live_odds(driver=driver)
live_odds

In [14]:
live_odds = repository.parse_live_odds(driver=driver)

# filter games to ones in live odds
games = [game for game in games if map_polymarket_name_to_fanduel_name[game.teams[0].name] in live_odds.keys()]

In [ ]:
live_odds

In [16]:
def listen_for_odds_change_return_relevant_games(live_odds_past_states, driver):

    while repository.parse_live_odds(driver=driver) == live_odds_past_states[-1]:
        time.sleep(1)

    else:
        current_odds = repository.parse_live_odds(driver=driver)
        changed_odds = {}
        
        for team, odds in current_odds.items():
            if team not in live_odds_past_states[-1] or live_odds_past_states[-1][team] != odds:
                changed_odds[team] = odds
                
        print(f"Changed odds: {changed_odds}")
        live_odds_past_states.append(current_odds)

    return changed_odds, live_odds_past_states

In [17]:
live_odds_past_states = []
live_odds_past_states.append(repository.parse_live_odds(driver=driver))

In [ ]:

while 1: 
    changed_odds, live_odds_past_states = listen_for_odds_change_return_relevant_games(live_odds_past_states, driver)
# change this to look for both teams
    games_to_trade = [game for game in games if map_polymarket_name_to_fanduel_name[game.teams[0].name] in changed_odds.keys() or map_polymarket_name_to_fanduel_name[game.teams[1].name] in changed_odds.keys()]
    print(f"games: {games_to_trade}")
    for game in games_to_trade:
        live_odds = repository.parse_live_odds(driver=driver)

        for team in game.teams:
            try:    
                cancel_res = polymarket_functions.cancel_single_order(client=client, order_id=team.last_order_id)
                print(f"cancel_res: {cancel_res['not_canceled'].values()}")
                if "matched orders can\'t be canceled" in cancel_res['not_canceled'].values():
                    print(f"Order for {team.name} was matched, setting last order filled time")
                    team.last_order_filled_time = time.time()
                
            except Exception as e:
                # {'not_canceled': {'0x944ce26545570852d917104dbccbd2af3c655a7cee5f1721940a0ad2e7aba0de': "matched orders can't be canceled"}, 'canceled': []}
                print(f"Error cancelling order for {team.name}: {e}")

            if live_odds[map_polymarket_name_to_fanduel_name[team.name]] is None:
                print(f"No odds for {team.name} breaking out of loop")
                break_condition = True
                break
            else: 
                break_condition = False

            # team.prediction_market_odds = Odds(probability=float(polymarket_functions.get_current_price(client, team.token_id)[team.token_id]['BUY']))       
            team.true_odds = Odds(american_odds=live_odds[map_polymarket_name_to_fanduel_name[team.name]])


        if break_condition:
            continue

        no_vig_dict = game.get_no_vig_odds(odds_type='true_odds')

        for team in game.teams:
            if team.last_order_filled_time and time.time() - team.last_order_filled_time < 600:
                print(f"Skipping {team.name} because it's been less than 10 minutes since last order was filled")
                continue

            if no_vig_dict is None:
                print(f"No no vig dict for {game.name}")
                continue

            team.no_vig_true_odds = no_vig_dict['team1_odds'] if team.name.lower() == no_vig_dict['team1_name'].lower() else no_vig_dict['team2_odds']

            desired_limit_price = team.get_limit_price_for_desired_expected_value(desired_expected_value=0.23)
            print(f"Creating limit order for {team.name} at {desired_limit_price}")

            usdc_amount_to_purchase = 10 / round(desired_limit_price, 3) 
            order_res = polymarket_functions.create_limit_order(client=client,
                                            side='buy',
                                            # minimum 1 minute security threshold, the 60 seconds won't count towards time. 
                                            seconds_to_last=60 + 22,
                                            usdcAmountToPurchase=usdc_amount_to_purchase, 
                                            token_id=team.token_id, 
                                            price=round(desired_limit_price, 3))
            print(f"order_res: {order_res}")
            team.last_order_id = order_res['orderID']
# {'errorMsg': '', 'orderID': '0x6a6155dc2b1f0546d9ca174ba4f9d46a26aa32af359d6199dea2e81f64589d7b', 'takingAmount': '', 'makingAmount': '', 'status': 'live', 'success': True}
            
            

notes from experiment. 

-need to set some sort of limit on total "to win" amount by team. this is because long shots can be a big investment even with just $10. 
-need to implement some sort of auto-sell, probably just copy this logic inverse it and put it in another thread. 
-need to look into the St. louis cardinals as it's showing as just St. for something. 
-need a way for new games to populate when tthey starrt